<a href="https://colab.research.google.com/github/lcbjrrr/quantai/blob/main/P99_Py_ASync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asynchronous programming

Asynchronous programming lets your program start tasks and then move on to other work without waiting for those tasks to finish immediately. It uses techniques like event loops, callbacks, promises, and async/await to handle the results of those tasks later, improving responsiveness and efficiency, especially for I/O-bound operations

# asyncio lib (and async and await)

This Python code demonstrates asynchronous programming with `asyncio`. The `background_task()` coroutine simulates a 3-second task using `asyncio.sleep()`. The `main()` coroutine starts the `background_task()` as a separate task using `asyncio.create_task()` and adds a callback to print a completion message. Importantly, `main()` continues executing other code (simulated by `asyncio.sleep(1)`) *without waiting* for `background_task()` to finish immediately. The `await asyncio.sleep(5)` later in `main()` ensures the background task has time to complete and trigger its callback. Finally, `asyncio.run(main())` within the `go()` function starts the asynchronous event loop, allowing the concurrent execution of `background_task()` and `main()`, showcasing how asynchronous code enables running multiple operations seemingly at the same time.


In [24]:
%%writefile async.py
import asyncio

async def background_task(time):
    await asyncio.sleep(time)
    print("Background task completed!")

async def main():
        print("Starting the main function...")
        t1=asyncio.create_task(background_task(2))  # Start the task without waiting
        t1.add_done_callback(lambda x: print("finished",x))
        t2=asyncio.create_task(background_task(1))  # Start the task without waiting
        t2.add_done_callback(lambda x: print("finished",x))
        print("Continuing the main function...")
        await asyncio.sleep(1)  # Simulate doing something else
        print("Waiting for the completion....")
        await asyncio.wait([t1,t2])
        print("Main function completed!")

def go():
  asyncio.run(main())

go()

Overwriting async.py


In [25]:
!python async.py

Starting the main function...
Continuing the main function...
Waiting for the completion....
Background task completed!
finished <Task finished name='Task-3' coro=<background_task() done, defined at /content/async.py:3> result=None>
Background task completed!
finished <Task finished name='Task-2' coro=<background_task() done, defined at /content/async.py:3> result=None>
Main function completed!


## Multi Threading

Threading is a way to achieve concurrency within a single program by dividing it into multiple independent execution paths called threads. These threads run concurrently within the same process, sharing the same memory space. This allows different parts of a program to execute seemingly simultaneously, improving performance for tasks that can be broken down into independent subtasks, especially on multi-core processors. However, because threads share memory, careful synchronization mechanisms like locks and mutexes are necessary to prevent race conditions and data corruption when multiple threads access and modify shared resources. Unlike asynchronous programming, which uses cooperative multitasking, threads are managed by the operating system (preemptive multitasking), meaning the OS decides when to switch between threads.

In [17]:
import threading
import time

def background_task(t):
    time.sleep(t)
    print(f"Background task completed after {t} seconds!")

def main():
    print("Starting the main function...")

    # Create and start threads for background tasks
    thread1 = threading.Thread(target=background_task, args=(2,))
    thread2 = threading.Thread(target=background_task, args=(1,))

    thread1.start()
    thread2.start()

    print("Continuing the main function...")
    time.sleep(1)  # Simulate doing something else
    print("Waiting for the completion....")

    # Wait for threads to finish
    thread1.join()
    thread2.join()

    print("Main function completed!")


main()

Starting the main function...
Continuing the main function...
Background task completed after 1 seconds!
Waiting for the completion....
Background task completed after 2 seconds!
Main function completed!


***When to use each one:***


| Feature | Threading | Async |
|---|---|---|
| Task Type | CPU-bound | I/O-bound |
| Parallelism | True parallelism (on multi-core) | Concurrency (not true parallelism in Python due to GIL) |
| Resource Usage | Higher (more memory and OS overhead) | Lower (less memory and OS overhead) |
| Complexity | More complex (requires careful synchronization) | Less complex (easier to avoid race conditions) |
| Best For | Computationally intensive tasks | Network and I/O operations |


## Multiprocessing

Multiprocessing in Python enables true parallelism by creating separate processes, each with its own memory space and Python interpreter, effectively bypassing the Global Interpreter Lock (GIL). This allows multiple CPU cores to execute code concurrently, significantly improving performance for CPU-bound tasks by distributing the workload across multiple processors, but at the cost of higher resource usage compared to threading.

In [18]:
import multiprocessing
import time

def worker(t):
    print(f"Process (sleeping for {t}s): Starting...")
    time.sleep(t)  # Simulate some work
    print(f"Process (slept for {t}s): Finished!")

def main():
    p1 = multiprocessing.Process(target=worker, args=(2,))  # Task with 2-second sleep
    p2 = multiprocessing.Process(target=worker, args=(1,))  # Task with 1-second sleep
    ps = [p1,p2]

    for p in ps: p.start()

    print("Continuing the main function...")
    time.sleep(1)  # Simulate doing something else
    print("Waiting for the completion....")

    for p in ps: p.join()

    print("All processes completed.")

main()

Process (sleeping for 2s): Starting...
Process (sleeping for 1s): Starting...
Continuing the main function...
Process (slept for 1s): Finished!
Waiting for the completion....
Process (slept for 2s): Finished!
All processes completed.
